<a href="https://colab.research.google.com/github/PrishaAggarwal/Conversational_AI_All_cources/blob/main/Accelerated_Data_Science/Lab/My_files_Colab/Convo_AI_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#deep neural network implementation
#first we are implementing we apply forward propagation
#binary wala implement kr rhe since we only know sigmoid activation function
#we need in order features,examples acc to our derivation in class

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from sklearn.datasets import load_breast_cancer
dataset=load_breast_cancer()
X=dataset.data
Y=dataset.target

In [ ]:
X.shape #hence it has 369 examples and 30 features

(569, 30)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
X_train=X_train.T
X_test=X_test.T
Y_train=Y_train.reshape(1,-1) #converted into one row
Y_test=Y_test.reshape(1,-1)

In [ ]:
#sabse phele define architecture of neural network
#kitni layers and kitne neural networks..chahe yeh kaam hyperprameters ka kaam hai
#tensorflow mein we have library mein jo tune krke de deti hai
#u might take diff number of layers here
#in tensorflow and pytorch bas yahi define krna hota hai architecture and u wont know forward ya backward propogation chl rha...no trainable parameters in  this layer
nn_arch=[
    {'layers_unit':30,'activation':'none'},#input layer doesnt do any activation so none #input layer has something to be trained
    {'layers_unit':5,'activation':'relu'},
    {'layers_unit':4,'activation':'relu'}, #4*5+4=24 trianable paramaters hai iske
    {'layers_unit':3,'activation':'relu'},
    {'layers_unit':1,'activation':'sigmoid'} #1 neuron since sigmoid .... agar 10 classes then 10 neurons with softmax function
]

In [ ]:
#aab har layer of weight bias define krenge
def initialize_parameters(nn_arch,seed_value):
  #we give seed value so that we give seed everytime and same parameters will be generated
  np.random.seed(seed_value)
  #iss nn_arch se it will know number of layers and functions
  parameters={}
  num_layers=len(nn_arch) #u will start from 1 to 5 and 5 will not be included
  #for 0th layer we dont need to define and 5th will be included in 1 to 5
  for l in range(1,num_layers):
    parameters['W'+str(l)]=np.random.randn(nn_arch[l]['layers_unit'],nn_arch[l-1]['layers_unit'])*0.01 #0.01 se multiplied since relu function suffers from gradient exploding problem
    #'w'  is weight of that layer and we have initialized the shape of weight as n,n-1
    #u cant initialize here to zero like in machine learning since varna sab neurons same ho jayenge i.e zero
    #sab neurons will become zero and activation will become zero and then non linear khtm
    #bias can be zero  initialize but not neurons
    #normally distributed inialization krni hai values called zavier initilization
    parameters['B'+str(l)]=np.zeros((nn_arch[l]['layers_unit'],1)) #bias initialized for lth layer with dimension l,1
  return parameters

In [ ]:
parameters=initialize_parameters(nn_arch,3)

In [ ]:
parameters['W1'].shape

(5, 30)

In [ ]:
def relu(z):
  return np.maximum(0,z)
  #agar z value is negative then gives 0 and otherwise same value

def sigmoid(z):
  return 1/(1+np.exp(-z))
#take any function but define that mathematical function here

In [ ]:
def forward_propagation(nn_arch,x,paramters): #kitni layers hai voh arch se pta lagega..intial paramters passing
  #x is input feature set and yeh given hota hai and calculate nhi hota so cache mein already yeh save kr lenge
  forward_cache={} #stores everything we need for backward from forward propagtion
  num_layers=len(nn_arch)
  forward_cache['A0']=x
  for l in range(1,num_layers):
    #we are seeing what are weight bias and activation function of layers
    W=parameters['W'+str(l)]
    B=paramters['B'+str(l)]
    activation=nn_arch[l]['activation']
    #now we have everything we need and we will compute the z value
    forward_cache['Z'+str(l)]=np.dot(W,forward_cache['A'+str(l-1)])+B #in general its W(l)A(l-1)+B(l)...this formula is only valid when dimensions are n0,n
    #relu mein less than 0 is 0 and more then 0 as it is...used more for images since negative values ko dead krna hai
    if activation=='relu':
      forward_cache['A'+str(l)]=relu(forward_cache['Z'+str(l)])
    elif activation=='sigmoid':
      forward_cache['A'+str(l)]=sigmoid(forward_cache['Z'+str(l)])
    #it should also return final calculated value since we have to find cost also jiske basis par hyperparamters are tuned toh predicted value bhi dekhni hai
    AL=forward_cache['A'+str(num_layers-1)] #last layer ki value is predicted value
    return AL,forward_cache
#we have just computed Z,A and osko save kr liya

In [ ]:
AL,forward_cache=forward_propagation(nn_arch,X_train,parameters)

KeyError: 'A4'

In [ ]:
parameters['B1'].shape

(5, 1)

In [ ]:
def compute_cost(y,AL):
  #we give actual value y and AL is predicted value
  n=y.shape[1]
  cost=(-1/n)*np.sum((y*np.log(AL+1e-8)+(1-y)*np.log(1-AL+1e-8))) #binary cross entropy cost function
  #added a very small value inside log i.e 1e-8 taki log 0 aake infinity na aa jaye
  #hyperparamter tuning ke time sometimes log 0 aa jata hai so osko avoid krne ke liye we did this
  return cost

In [ ]:
compute_cost(Y_train,AL)

NameError: name 'AL' is not defined

In [ ]:
#lets start backward propagation
#we have compute dA(l) khud compute krke dena padega of  last layer
#aise hi we have derived for all other parameters in backward propogation
#aise hi we find dw(l) and db(l) nikalte hai

In [ ]:
def sigmoid_backward(dA_prev,Z): #g'(z) is derivation of sigmoid function w.r.t z
  #when g is diff then g' i.e activation function alag hoga in backward
  dact=sigmoid(Z)*(1-sigmoid(Z)) #dact is derivative of activation function
  return dA_prev*dact

In [ ]:
def relu_backward(dA_prev,Z):
  #0 par bhi we take 0 kyuki 0 neuron aane ki probability is very less
  dA_prev=np.array(dA_prev,copy=True) #pichla dA aage chahiye hoga and hence voh erase na ho
  dA_prev[Z<0]=0
  #when z is greater than 1 then it will get multiplied by 1 i.e da prev vhi rahega but 0 wale case mein it becomes zero so
  return dA_prev

In [ ]:
def backward_propagation(nn_arch,y,AL,forward_cache,parameters): #we need y for dA phele wala to calculate, need w from parameters, and activation functions from forward cache
  grads={} #gradients #loss function ka derivative w.r.t wt and bias store krenge ismein
  num_layers=len(nn_arch)
  dA_prev=(y-AL)/(AL*(1-AL)) #da_prev pheli baari mein hum compute krke denge..binary classification mein da_prev ka yeh formula hai varna for multiclass its diff
  n=y.shape[1]
  for l in reversed(range(1,num_layers)):
    #backward ulti chlti hai 4,3,2,1 chlegi
    #har layer ke 4 values nikalni hai and onmein jo cheeze use hogi voh phele nikalo
    z_curr=forward_cache['Z'+str(l)]
    A_prev=forward_cache['A'+str(l-1)]
    W_curr=parameters['W'+str(l)]
    activation=nn_arch[l]['activation']
    if activation=='relu':
      dZ=(1/n)*relu_backward(dA_prev,z_curr) #cost mein 1/n hota hai so derivation mein bhi 1/n aayega
      grads['dW'+str(l)]=np.dot(dZ,A_prev.T)
      grads['dB'+str(l)]=np.sum(dZ,axis=1,keepdims=True) #agar addition kr rhe then column is return ho keepdims=True
      dA_prev=np.dot(W_curr.T,dZ)
    elif activation=='sigmoid':
      dZ=(1/n)*sigmoid_backward(dA_prev,z_curr)
      grads['dW'+str(l)]=np.dot(dZ,A_prev.T)
      grads['dB'+str(l)]=np.sum(dZ,axis=1,keepdims=True)
      dA_prev=np.dot(W_curr.T,dZ)
    return grads

In [ ]:
grads=backward_propagation(nn_arch,Y_train,AL,forward_cache,parameters)

NameError: name 'AL' is not defined

In [ ]:
def update_parameters(paramters,grads,lr,nn_arch): #optimizing techniques using
#we are using here gradient descent
  for l in range(1,len(nn_arch)):
    parameters['W'+str(l)]=parameters['W'+str(l)]-lr*grads['dW'+str(l)]
    parameters['B'+str(l)]=parameters['B'+str(l)]-lr*grads['dB'+str(l)]
  return parameters
  #gradient descent neural networks mein not used since zig zag curve ban jata hai and hamare parameters ache se optimize nhi hote

In [ ]:
update_parameters(parameters,grads,0.01,nn_arch)

NameError: name 'grads' is not defined

In [ ]:
def training(x,y,nn_arch,lr,iterations): #ismein yeh sab ekathe krenge
#model.compile yahi training wala part kr rha hai
  parameters=initialize_parameters(nn_arch,3) #har layer ka weight and bias initialized mil gya
  costs=[] #list of cost
  for i in range(iterations): #iterations is hyperparameter
    AL,forward_cache=forward_propagation(nn_arch,x,parameters) #phele z and activation compute hogi using forward propogation for each iterations
    cost=compute_cost(y,AL) #current values of weight and bias se cost computed
    grads=backward_propagation(nn_arch,y,AL,forward_cache,parameters) #gradients computed
    parameters=update_parameters(parameters,grads,lr,nn_arch)
    #after every 100 iterations print the cost to check it regularly
    if(i%100==0):
      print('cost after '+str(i)+' iterations is '+str(cost))
      costs.append(cost) #plot krenge yeh cost and osse hyperparameter tuning krenge and also see if curve is converging or not
  return costs,parameters,AL #we return predicted values since testing ke time predicted value chahiye hogi

In [ ]:
costs,parameters,predicted_values=training(X_train,Y_train,nn_arch,0.01,10000)

KeyError: 'A4'

In [ ]:
#plot this cost
import matplotlib.pyplot as plt
plt.plot(costs)
#converging costs but zig zag curves coming
#this is why gradient descent not used in deep learning

NameError: name 'costs' is not defined

In [ ]:
#we have weights after training and X _test also
#forward propagation se we find predicted value
#akhri neuron ki value is our ans
Y_predicted,forward_cache=forward_propagation(nn_arch,X_test,parameters)
#testing ke time we dont need forward cache but we wrote since forward wala function it returns

KeyError: 'A4'

In [ ]:
#values are between 0 to 1 since sigmoid values hai
Y_predicted[Y_predicted<=0.5]=0
Y_predicted[Y_predicted>0.5]=1
Y_predicted

NameError: name 'Y_predicted' is not defined

In [ ]:
np.sum(Y_predicted==Y_test)/len(Y_test)

NameError: name 'Y_predicted' is not defined

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_predicted.reshape(-1,1),Y_test)
#in built functions demand that column vector mein ho feature so we reshape it

NameError: name 'Y_predicted' is not defined

In [ ]:
best_model=[
    'accuracy':-np.inf, #we need accuracy high so anything more than -inf is better
    'cost':np.inf, #we need minimum cost so highest se everything is better
    'lr':None,
    'iteration':None,
    'predicted_value':None,
    'parameters':None
    ]

SyntaxError: invalid syntax (ipython-input-613588809.py, line 2)

In [ ]:
#HYPERPARAMETER TUNING
#Basically grid search hi hoti hai
epochs=[100,1000,10000] #we take a list of epochs
alphas=[1e-4,1e-3,1e-2,1e-1]
for iteration in epochs:
  for lr in alphas:
    #aab har combination par we call training function
    costs,parameters,AL=training(X_train,Y_train,nn_arch,lr,iteration) #ismein use krte vakat humne cost band krdi since baar baar printing
    #aab best model kaise nikalenge ?
    #upar ek best model ki dictionary we take with accuracy and cost ke basis we update best model
    Y_predicted,_=forward_propagation(nn_arch,X_test,parameters) #trainign se best parameters nikalkr y ko predict kra
    Y_predicted[Y_predicted<=0.5]=0
    Y_predicted[Y_predicted>0.5]=1
    acc=accuracy_score(Y_predicted.reshape(-1,1),Y_test)
    if (acc>best_model['accuracy']) or (acc==best_model['accuracy'] and costs[-1]<best_model['cost']):
      #[-1]costs mein last cost value jo best cost nikalkr aayi hai
      best_model.update({
          'accuracy':acc,
          'cost':costs[-1],
          'lr':lr,
          'iteration':iteration,
          'predicted_value':AL,
          'parameters':parameters
       })

KeyError: 'A4'

In [ ]:
best_model

NameError: name 'best_model' is not defined